In [3]:
# ideas for a source test
import cobra
import pandas as pd

from refinegems.utility.io import load_model

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [4]:
test_model_path = '/Users/brune/Documents/11_Test_Data/test_SPECIMEN/thesis/Kp_std/03_refinement/step4-smoothing/Kp_std_smooth.xml'
test_model = load_model(test_model_path,'cobra')


-----
### Plot the basic analysis

- one model  -  done more or less
- collection of models

#### Plot for a single report

In [ ]:
from refinegems.utility.io import load_model
from refinegems.classes.reports import ModelInfoReport
import matplotlib.pyplot as plt

test_model_path = '/Users/brune/Documents/11_Test_Data/test_SPECIMEN/thesis/Kp_std/03_refinement/step4-smoothing/Kp_std_smooth.xml'
test_model = load_model(test_model_path,'cobra')

rep = ModelInfoReport(test_model)
fig = rep.visualise()

plt.show()

------
# EGC

In [5]:
import cobra
import argparse
from tqdm import tqdm
from multiprocess import Pool
from functools import partial
from itertools import product

In [ ]:


def main(input_path, output_path, change_model):
    model = cobra.io.read_sbml_model(input_path)

    # check for egcs before hand -> is solution necessary? -> increases performance -> less egcs to check
    egc_reactions, obj_vals = find_egcs(model)

    # resolve egcs
    results = find_mods_resolve_egcs(model, present_egcs=egc_reactions)

    # bring results to condensed format
    condensed_results = condense_results(results)

    # get all possible solutions
    all_solutions = get_all_solutions(condensed_results)

    # find minimal changes
    score, best_score_idx = find_minimal_changes(all_solutions)

    # print best result
    best_solution = all_solutions[best_score_idx]
    print(f"One optimal solution is: {best_solution}")
    print(f"With a score of: {score[best_score_idx]}")

    # print other results?
    # this is heuristic...

    # TODO save changes
    if output_path and change_model:
        out_model = model
        out_model = apply_modifications(out_model, best_solution)

        # TODO check whole model again in the end to verfy we havent established anyother egcs
        check_egc_reactions, obj_vals = find_egcs(out_model)
        if not check_egc_reactions:
            cobra.io.write_sbml_model(out_model, output_path)
            print(f"Removed EGCs from model.\nModel is saved to {output_path}.")

    elif change_model and not output_path:
        print("No output path provided. Model will not be changed.")

    elif output_path and not change_model:
        print(f"'--change' flag set to {change_model}. Please set to 'TRUE' to change and save the model.")

    else:
        print("Calculated one optimal solution.")

    return 1


--------

In [6]:
from refinegems.classes.medium import Medium
from refinegems.curation.biomass import test_biomass_presence
from refinegems.analysis.growth import set_bounds_to_default

from typing import Literal
import warnings

In [7]:
# needed information

# namespace to use
namespace = 'BiGG'

# compartment to search for metabolites
compartment = 'c'
compartment_2 = 'p'

# @TODO
# NOTE: Ammonia in DB is NH3 - this should be NH4 - change it?
# reactions equations to add / factors etc.
DISSIPATION_RXNS = {
    "ATP": {"ATP [Adenosine triphosphate]": -1, "Water [H2O]": -1, "ADP [Adenosine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "CTP": {"CTP [Cytidine triphosphate]": -1, "Water [H2O]": -1, "CDP [Cytidine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "GTP": {"GTP [Guanosine triphosphate]": -1, "Water [H2O]": -1, "GDP [Guanosine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "UTP": {"UTP [Uridine triphosphate]": -1, "Water [H2O]": -1, "UDP [Uridine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "ITP": {"ITP [Inosine triphosphate]": -1, "Water [H2O]": -1, "IDP [Inosine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "NADH": {"NADH [reduced Nicotinamide adenine dinucleotide]": -1, "Hydrogen [H(+)]": 1, "NAD [oxidized Nicotinamide adenine dinucleotide]": 1},
    "NADPH": {"NADPH [reduced Nicotinamide adenine dinucleotide phosphate]": -1, "Hydrogen [H(+)]": 1, "NADP [oxidized Nicotinamide adenine dinucleotide phosphate]": 1},
    "FADH2": {"FADH2 [reduced Flavin adenine dinucleotide]": -1, "Hydrogen [H(+)]": 2, "FAD [oxidized Flavin adenine dinucleotide]": 1},
    "FMNH2": {"FMNH2 [reduced Flavin mononucleotide]": -1,  "Hydrogen [H(+)]": 2, "FMN [oxidized Flavin mononucleotide]": 1},
    "Q8H2": {"Ubiquinone-8": -1, "Hydrogen [H(+)]": 2, "Ubiquinol-8": 1},
    "MQL8": {"Menaquinone-8": -1, "Hydrogen [H(+)]": 2, "Menaquinol-8": 1},
    "DMMQL8": {"2-Demethylmenaquinone-8": -1, "Hydrogen [H(+)]": 2, "2-Demethylmenaquinol-8": 1},
    "ACCOA": {"Acetyl-CoA": -1, "Water [H2O]": -1, "Hydrogen [H(+)]": 1, "Acetate [Acetic acid]": 1, "Coenzyme A": 1},
    "GLU": {"D-Glucose": -1, "Water [H2O]": -1, "2-Oxoglutarate [Oxoglutaric acid]": 1, "Ammonia": 1, "Hydrogen [H(+)]": 2},
    "PROTON": {"Hydrogen [H(+)]": 1, "Hydrogen [H(+)] transported": -1}
}

ECG_SCORING_MATRIX = {'MR':1, 'RB':3, 'RF':3,'RM':6}

# 1. if reaction is not reversible -> make reaction reversible (MR)
# 2. limit backward reaction (RB)
# 3. limit forward reaction (RF)
# 4. "delete" reaction by setting fluxes to 0 (RM)



In [14]:
# find EGC
# --------

def add_DISSIPATIONRXNS(model: cobra.Model,
                        namespace:Literal['BiGG']='BiGG',
                        compartment:list=['c','p']) -> cobra.Model:
    
    def check_metab_integration(metabolites: dict[str: int], model: cobra.Model,
                            metab_info:Medium, namespace:Literal['BiGG']='BiGG',compartment:list=['c','p']) -> None|dict:

        found_ids = {}

        c1_metab = [_.id for _ in model.metabolites if _.compartment == compartment[0]]
        c2_metab = [_.id for _ in model.metabolites if _.compartment == compartment[1]]

        for meta in list(metabolites.keys()):
            # get metabolite database annotations
            pos_ids = metab_info.substance_table[metab_info.substance_table['name']==meta][['db_type','db_id']]

            # check namespace availability 
            if not any(namespace in _ for _ in list(pos_ids['db_type'])):
                return None
            # check metabolite availability in model
            id = pos_ids[pos_ids['db_type'].str.contains(namespace)]['db_id']
            for i in id:
                match namespace:
                    # BiGG ID needs the compartment with a '_' as suffix
                    case 'BiGG':
                        if len(metabolites) == 2: # special case proton
                            i_p = i + '_' + compartment[1]
                            i_c = i + '_' + compartment[0]
                            if i_p not in c2_metab or i_c not in c1_metab:
                                return None
                            else:
                                found_ids[i_p] = metabolites['Hydrogen [H(+)] transported']
                                found_ids[i_c] = metabolites['Hydrogen [H(+)]']
                                return found_ids
                        else:
                            i += '_' + compartment[0]
                    # No namespacce given
                    case _:
                        mes = f'Unknown namespace or no namespace given: {namespace}'
                        warnings.warn(mes)
                        return None

                if i not in c1_metab:
                    return None
                else:
                    found_ids[i] = metabolites[meta]
                    break 

        return found_ids

    # retrieve information about dissipation reaction metabolites
    metab_info = Medium('dissipation reaction metabolites')
    metab_info.add_subset('DiReM')
    
    # add dissipations reactions to model
    for name, metabolites in DISSIPATION_RXNS.items():
        if "DISSI_"+name not in model.reactions:
            ids_in_namespace = check_metab_integration(metabolites, model,
                                       metab_info, namespace,compartment)
            if ids_in_namespace:
                rea = cobra.Reaction(name=name, id="DISSI_"+name)
                model.add_reactions([rea])
                rea.add_metabolites(ids_in_namespace)

    return model


def limit_bounds(model: cobra.Model) -> cobra.Model:
    """Limits upper and lower bounds of
    exchange reactions to (0, 0)
    reversible reactions to (-1, 1)
    irreversible reactions to (0, 1)

    excludes dissipation reactions

    Args:
        model (cobra.Model): cobrapy model

    Returns:
        cobra.Model: cobrapy model
    """
    external_comp = cobra.medium.find_external_compartment(model)
    # set fluxes for each reaction within model
    for rea in model.reactions:
        # except dissipation reactions
        if "DISSI_" in rea.id:  
            continue
        # turn off exchange reactions
        elif cobra.medium.is_boundary_type(rea,'exchange',external_comp):
            rea.bounds = (0.0, 0.0)
        # limit reversible reactions to [-1, 1] -> flux 0.1
        elif rea.reversibility: 
            rea.bounds = (-1.0, 1.0)
        # limit irreversible reactions to [0, 1] -> flux 0.1
        else: 
            rea.bounds = (0.0, 1.0)

    return model



def find_egcs(model: cobra.Model) -> tuple[dict, dict]:
    """Checks a cobra.Model for 15 Energy Generating Cycles (EGCs) specified in 
    DISSIPATION_RXN by adding the dissipation reactions turning off exchange reactions
    and limit other reactions to 0.1% flux.

    Based on Fritzemaier et al. (2017, https://doi.org/10.1371/journal.pcbi.1005494)    

    Args:
        model (cobra.Model): GEM read in with cobra

    Returns:
        tuple[dict, dict]: (egc_reactions, objective_values)
    """
    # add 15 energy dissipation reactions
    with model as mod_model:  

        # ensure no model modifications
        mod_model = add_DISSIPATIONRXNS(mod_model)

        # set fluxes for each reaction within model
        mod_model = limit_bounds(mod_model)

        # set each dissipation reaction as objective & optimize
        egc_reactions = {}
        obj_vals = {}
        for name in DISSIPATION_RXNS.keys():
            rea_id = "DISSI_"+name
            if rea_id in mod_model.reactions:
                mod_model.objective = "DISSI_"+name
                solution = mod_model.optimize()
                fluxes = solution.fluxes
                objval = solution.objective_value

                if objval > 0.0:  # optimization > 0 --> EGC detected
                    obj_vals[name] = objval
                    egc_reactions[name] = {}

                    for rea, flux in fluxes.items():
                        # cutoff for flux 1.0e-10 (=no growth) 
                        if  (flux > 1.0e-10 or flux < -1.0e-10) and not rea.startswith("DISSI"):
                            egc_reactions[name][rea] = flux

    return (egc_reactions, obj_vals)


# find solutions for EGCs
# -----------------------

# ...........................................................
# condense, shorten etc. - way to bulky currently 
# OPTIMIZE
# + very dangerous naming
# ...........................................................
# naming dangerous, as it returns True when EGC is NOT present
def check_single_egc(model: cobra.Model, egc: str) -> bool:
    """Checks if an EGC is NOT present. E.g. objective value of model optimized
    for DISSI_ATP > 0.

    Args:
        model (cobra.Model): cobra Model
        egc (str): Name of the dissipation reaction, e.g. ATP

    Returns:
        bool: True if objective value == 0.0. False if objective value > 0.0.
    """
    with model:  # need to add dissi reas & limit bounds each time
        rea_id = "DISSI_"+egc
        # add dissipation rxns -> needed to added each time
        model = add_DISSIPATIONRXNS(model)
        # set fluxes for each reaction within model
        mod_model = limit_bounds(model)

        if rea_id in mod_model.reactions:
            mod_model.objective = rea_id
            solution = mod_model.optimize()
            if solution.objective_value > 0.0:
                return False
            else:
                return True

# why rich? 
# ......            
def sim_growth_richMedium(model: cobra.Model, bof:str, threshold: float = 0.0) -> bool:
    """Simulates growth of a model in rich medium. Sets the model objective to
    'Growth' before simulation. Returns True if growth exceeds threshold.

    Args:
        model (cobra.Model): cobra Model
        threshold (float, optional): Threshold which objective value has to exceed.
        Defaults to 0.0.

    Returns:
        bool: True if obj_val >= threshold. False otherwise.
    """

    with model as mod_model:
        # cleanup bounds
        set_bounds_to_default(mod_model)
        # set model objective to growth reaction
        mod_model.objective = bof
        solution = mod_model.optimize()

        if solution.objective_value > threshold:
            return True

    return False


def check_egc_growth(model: cobra.Model, egc: str, bof:str) -> bool:
    """Checks a model if a specified EGC is NOT present & the model can grow in rich medium.

    Args:
        model (cobra.Model): GEM loaded with cobrapy
        egc (str): Name of the EGC

    Returns:
        bool: True == EGC missing & model can grow; False otherwise
    """
    if check_single_egc(model, egc):
        if sim_growth_richMedium(model, bof):
            return True
        else:
            return False
    else:
        return False

# ...........................................................
    

def test_modifications(reaction: cobra.Reaction, model: cobra.Model, present_egc: dict, results: dict, bof: str) -> dict:
    """Tries four cases for a Reaction:
    1. if reaction is not reversible -> make reaction reversible (MR)
    2. limit backward reaction (RB)
    3. limit forward reaction (RF)
    4. "delete" reaction by setting fluxes to 0 (RM)
    -> for each case the EGCs which are present in the model are checked if they are removed
    -> if EGCs are removed we check if the model still grows on optimal medium
    => When both limitations are True reaction is saved to corresponding dictionary

    Args:
        reaction (cobra.Reaction): Reaction from a cobra.Model
        model (cobra.Model): The corresponding GEM loaded with cobrapy
        present_egc (dict): Dictionary of present EGCs {"egc": {}} -> EGCs are keys
        results (dict): "Empty" dictionary of dictionary formated of results {"egc": {"MR":[], "RB":[], "RF":[], "RM":[]}}

    Returns:
        dict: {"egc": {"MR":[potential_solutions],
                       "RB":[potential_solutions],
                       "RF":[potential_solutions],
                       "RM":[potential_solutions]}}
    """
    for egc in present_egc:  # skip egc which are not present -> increase performance
        with model:
            if not reaction.reversibility:  # skip if reaction is already reversible
                reaction.bounds = (-1000.0, 1000.0)  # irreversible -> reversible
                if check_egc_growth(model, egc, bof):
                    results[egc]["MR"].append(reaction.id)  # MR = make reversible

            reaction.bounds = (0, 1000.0)  # limit backward reaction
            if check_egc_growth(model, egc, bof):
                results[egc]["RB"].append(reaction.id)   # Remove Backward (RB) reaction

            reaction.bounds = (-1000.0, 0)  # limit forward reaction
            if check_egc_growth(model, egc, bof):
                results[egc]["RF"].append(reaction.id)   # Remove Forward (RF) reaction

            reaction.bounds = (0, 0)  # "delete" reaction
            if check_egc_growth(model, egc, bof):
                results[egc]["RM"].append(reaction.id)   # ReMove reaction

    return results


# @TEST - NOTE: changed global param to local, make sure it works correctly
# @TEST good default for limit and chunksize
# params not optimal and runtime options need rechecking
def find_mods_resolve_egcs(model: cobra.Model, present_egcs: dict,
                           limit:int=8, chunksize:int=5) -> dict:
    """Find the modifications to reactions in a cobra.Model and returns these in a dictionary.
    Splits the modification check in multiple processes.
    Run-time around 20 min for 1300 reactions (roughly 0.92s/reaction).

    Args:
        model (cobra.Model): input GEM

    Returns:
        dict: Dictionary of potential modifications to resolve EGCs
              {"egc": {"MR":[potential_solutions],
                       "RB":[potential_solutions],
                       "RF":[potential_solutions],
                       "RM":[potential_solutions]}}
    """
    results = {
            "ATP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "CTP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "GTP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "UTP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "ITP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "NADH": {"MR": [], "RB": [], "RF": [], "RM": []},
            "NADPH": {"MR": [], "RB": [], "RF": [], "RM": []},
            "FADH2": {"MR": [], "RB": [], "RF": [], "RM": []},
            "FMNH2": {"MR": [], "RB": [], "RF": [], "RM": []},
            "Q8H2": {"MR": [], "RB": [], "RF": [], "RM": []},
            "MQL8": {"MR": [], "RB": [], "RF": [], "RM": []},
            "DMMQL8": {"MR": [], "RB": [], "RF": [], "RM": []},
            "ACCOA": {"MR": [], "RB": [], "RF": [], "RM": []},
            "GLU": {"MR": [], "RB": [], "RF": [], "RM": []},
            "PROTON": {"MR": [], "RB": [], "RF": [], "RM": []}
            }
    output_list = []

    print("_____________________________________")
    print(f"Try to resolve the following EGCs: {[egc for egc in present_egcs]} \nThis might take a while...")

    # try to find BOF
    pos_bofs = test_biomass_presence(model)
    if pos_bofs:
        bof = pos_bofs[0]
    else:
        mes = 'No growth or biomass objectuve function in model. Cannot solve EGCs.'
        raise KeyError(mes)

    # might limit processes with Pool(process=limit) -> otherwise it consumes all cores
    # limit to half of machine cores -> at least for me no speed increment with more cores
    with Pool(processes=limit) as pool:
                  
        # partial -> creates function with fixed variables to call with each iteration
        # needed since pool.imap cannot do that
        part_test_mods = partial(test_modifications,
                                model=model,
                                present_egc=present_egcs,
                                bof=bof,
                                results=results)
        # increment in chunksize will reduce computation time -> but progressbar update also...
        for res in list(tqdm(pool.imap(func=part_test_mods, iterable=model.reactions, chunksize=chunksize),
                        total=len(model.reactions),
                        desc="Resolve EGCs")):
            
            output_list.append(res)
        

    # merge output_list to the final results
    for output_dict in output_list:
        for egc, modifications in output_dict.items():
            for mod, reactions in modifications.items():
                results[egc][mod] = list(set(results[egc][mod] + reactions))

    return results


# find (one of the) best solution(s)
# ----------------------------------

# ...?
def condense_results(results: dict) -> dict:
    """Condenses the output from find_mods_resolve_egcs().
    For easier handling of the dictionary.

    Args:
        results (dict): result from find_mods_resolve_egcs()

    Returns:
        dict: condensed dictionary without empty entries.
    """
    return {key: {inner_key: inner_value for inner_key, inner_value in value.items() if inner_value} for key, value in results.items() if any(value.values())}


def get_all_solutions(result: dict) -> list:
    """Builds all possible solutions to resolve all EGCs.
    Output is in form of:
    {modification: {reaction: [EGCs]}}

    Args:
        result (dict): output from condense_results

    Returns:
        list: list of dictionaries, each representing a possible solution to resolve all EGCs.
    """
    def unflatten(pairs: tuple) -> dict:
        """Intern function: unflattens the solution produced in here.

        Args:
            pairs (tuple): possible solutions in flatted form

        Returns:
            dict: Output dictionary of get_all_solutions()
        """
        m = {}
        for egc, (mod, react) in pairs:
            try:
                m[mod][react].append(egc)
            except KeyError:
                try:
                    m[mod].update({react: [egc]})
                except KeyError:
                    m[mod] = {react: [egc]}
        return m

    def paths(flat_solution, needle):
        """Intern function to get the paths of a possible solution for an EGC (needle).

        Args:
            flat_solution (_type_): _description_
            needle (_type_): _description_

        Returns:
            _type_: _description_
        """
        return [(mod, react) for (egc, mod, react) in flat_solution if egc == needle]

    def flatten(egcs):
        """Intern function to flatten the input dictionary.

        Args:
            egcs (_type_): _description_

        Returns:
            _type_: _description_
        """
        return [(egc, mod, react) for egc, mods in egcs.items() for mod, reacts in mods.items() for react in reacts]

    flat_result = flatten(result)
    egcs = list(set([egc for egc, mod, react in flat_result]))
    flat_solutions = []

    for prod in product(*[[(egc, modreact) for egc in egcs for modreact in paths(flat_result, egc)]]):
        flat_solutions.append(list(prod))

    return [unflatten(solution) for solution in flat_solutions]


def find_minimal_changes(all_solutions: list) -> tuple[list, int]:
    """Calculates the score and returns a list of all scores and the index of the minimal score.

    Args:
        all_solutions (list): All possible solutions in form of a list of dictionaries.

    Returns:
        tuple: tuple of (list, int) list == all scores, int == index of lowest score
    """
    sums = []
    for sol in all_solutions:
        counter = 0
        for mod, reas in sol.items(): 
                counter += ECG_SCORING_MATRIX[mod] * len(reas)
        sums.append(counter)
        
    return sums, sums.index(min(sums))


# change the model
# ----------------

def apply_modifications(model: cobra.Model, solution: dict) -> cobra.Model:
    """Apply the modifications to reactions in solution to the model.
    4 modifications are possible:
    "RM" -> removes the reaction
    "RB" -> removes the backwards reaction
    "RF" -> removes the forward reaction
    "MR" -> makes reaction reversible

    Args:
        model (cobra.Model): Input model
        solution (dict): Best solution from calculation before.

    Returns:
        cobra.Model: modified cobra.Model.
    """
    for mod, react_list in solution.items():
        for react in react_list.keys():
            reaction = model.reactions.get_by_id(react)
            if type(reaction) is cobra.Reaction:
                if mod == "RM":
                    reaction.delete()
                    print(f"{reaction.id} is removed.")
                elif mod == "RB":
                    reaction.bounds = (0.0, 1000.0)
                    print(f"Backward reaction is removed from {reaction.id}.")
                elif mod == "RF":
                    model = reaction.bounds = (1000.0, 0.0)
                    print(f"Forward reaction is removed from {reaction.id}.")
                elif mod == "MR":
                    model = reaction.bounds = (1000.0, 1000.0)
                    print(f"{reaction.id} is now reversible.")
                else:
                    print(f"{mod} is no viable modification... Something went wrong.")



In [9]:
test_egc_path = '/Users/brune/Downloads/iJN746.xml'
test_egc = load_model(test_egc_path,'cobra')

test_model_path = '/Users/brune/Documents/11_Test_Data/test_SPECIMEN/thesis/Kp_std/03_refinement/step4-smoothing/Kp_std_smooth.xml'
test_model = load_model(test_model_path,'cobra')

if len(find_egcs(test_model)) == 0:
    print('t')

In [ ]:
with test_egc as t:
    egc_reactions, obj_vals = find_egcs(t)
    results = find_mods_resolve_egcs(t, present_egcs=egc_reactions, chunksize=16)

results

_____________________________________
Try to resolve the following EGCs: ['ATP', 'CTP', 'GTP', 'UTP', 'NADH', 'NADPH', 'FADH2', 'FMNH2', 'Q8H2', 'ACCOA', 'PROTON'] 
This might take a while...


Resolve EGCs: 100%|██████████| 1054/1054 [15:18<00:00,  1.15it/s]


{'ATP': {'MR': [],
  'RB': ['ALDD2x_copy2', 'ACKr'],
  'RF': ['ALDD2x_copy2', 'ACKr'],
  'RM': ['ALDD2x_copy2', 'ACKr']},
 'CTP': {'MR': [],
  'RB': ['ALDD2x_copy2', 'ACKr'],
  'RF': ['ALDD2x_copy2', 'ACKr'],
  'RM': ['ALDD2x_copy2']},
 'GTP': {'MR': [],
  'RB': ['ALDD2x_copy2', 'ACKr'],
  'RF': ['ALDD2x_copy2', 'ACKr'],
  'RM': ['ALDD2x_copy2', 'ACKr']},
 'UTP': {'MR': [],
  'RB': ['ALDD2x_copy2', 'ACKr'],
  'RF': ['ALDD2x_copy2', 'ACKr'],
  'RM': ['ALDD2x_copy2']},
 'ITP': {'MR': [], 'RB': [], 'RF': [], 'RM': []},
 'NADH': {'MR': ['MMTSAO',
   'PSD181',
   'OXOADLR',
   'NTD3',
   'PPPGO',
   'EX_ga_e',
   'DXPRIi',
   'NTD2pp',
   'ASPO6',
   'HEX1',
   'MGCH',
   'THDPS',
   'AGPAT161',
   'PHAPC100',
   'TRPS2',
   'METAT',
   'DM_dad_5_c',
   '3HAD120',
   'EX_phe__L_e',
   'HMPK1',
   'EX_hdca_e',
   'EX_3oxoadp_e',
   'APENTAMAH',
   'ALCDkt',
   'GUAD',
   'RNDR2',
   'NADH5',
   'CBPS',
   'PGPP160pp',
   'AGPAT120',
   '3OXCOAT',
   'EX_glcn_e',
   'EX_tyr__L_e',
   'UPP3S',

------

## EGCs - new try

In [1]:
from refinegems.analysis.growth import MIN_GROWTH_THRESHOLD
from refinegems.classes.medium import Medium
from refinegems.curation.biomass import test_biomass_presence
from refinegems.analysis.growth import set_bounds_to_default
from refinegems.utility.io import load_model

import cobra
import pandas as pd
from tqdm import tqdm
from multiprocess import Pool
from functools import partial
from itertools import product

from typing import Literal
import warnings

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# needed information

# namespace to use
namespace = 'BiGG'

# compartment to search for metabolites
compartment = 'c'
compartment_2 = 'p'

# @TODO
# NOTE: Ammonia in DB is NH3 - this should be NH4 - change it?
# reactions equations to add / factors etc.
DISSIPATION_RXNS = {
    "ATP": {"ATP [Adenosine triphosphate]": -1, "Water [H2O]": -1, "ADP [Adenosine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "CTP": {"CTP [Cytidine triphosphate]": -1, "Water [H2O]": -1, "CDP [Cytidine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "GTP": {"GTP [Guanosine triphosphate]": -1, "Water [H2O]": -1, "GDP [Guanosine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "UTP": {"UTP [Uridine triphosphate]": -1, "Water [H2O]": -1, "UDP [Uridine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "ITP": {"ITP [Inosine triphosphate]": -1, "Water [H2O]": -1, "IDP [Inosine diphosphate]": 1, "Hydrogen [H(+)]": 1, "Phosphate [PO4(3-)]": 1},
    "NADH": {"NADH [reduced Nicotinamide adenine dinucleotide]": -1, "Hydrogen [H(+)]": 1, "NAD [oxidized Nicotinamide adenine dinucleotide]": 1},
    "NADPH": {"NADPH [reduced Nicotinamide adenine dinucleotide phosphate]": -1, "Hydrogen [H(+)]": 1, "NADP [oxidized Nicotinamide adenine dinucleotide phosphate]": 1},
    "FADH2": {"FADH2 [reduced Flavin adenine dinucleotide]": -1, "Hydrogen [H(+)]": 2, "FAD [oxidized Flavin adenine dinucleotide]": 1},
    "FMNH2": {"FMNH2 [reduced Flavin mononucleotide]": -1,  "Hydrogen [H(+)]": 2, "FMN [oxidized Flavin mononucleotide]": 1},
    "Q8H2": {"Ubiquinone-8": -1, "Hydrogen [H(+)]": 2, "Ubiquinol-8": 1},
    "MQL8": {"Menaquinone-8": -1, "Hydrogen [H(+)]": 2, "Menaquinol-8": 1},
    "DMMQL8": {"2-Demethylmenaquinone-8": -1, "Hydrogen [H(+)]": 2, "2-Demethylmenaquinol-8": 1},
    "ACCOA": {"Acetyl-CoA": -1, "Water [H2O]": -1, "Hydrogen [H(+)]": 1, "Acetate [Acetic acid]": 1, "Coenzyme A": 1},
    "GLU": {"D-Glucose": -1, "Water [H2O]": -1, "2-Oxoglutarate [Oxoglutaric acid]": 1, "Ammonia": 1, "Hydrogen [H(+)]": 2},
    "PROTON": {"Hydrogen [H(+)]": 1, "Hydrogen [H(+)] transported": -1}
}

ECG_SCORING_MATRIX = {'MR':1, 'RB':3, 'RF':3,'RM':6}

#### Find the EGCs

NOTE: combination possibility of `find_egcs` with `find_egcs_only`

In [3]:
# find EGC
# --------

def add_DISSIPATIONRXNS(model: cobra.Model,
                        namespace:Literal['BiGG']='BiGG',
                        compartment:list=['c','p']) -> cobra.Model:
    
    def check_metab_integration(metabolites: dict[str: int], model: cobra.Model,
                            metab_info:Medium, namespace:Literal['BiGG']='BiGG',compartment:list=['c','p']) -> None|dict:

        found_ids = {}

        c1_metab = [_.id for _ in model.metabolites if _.compartment == compartment[0]]
        c2_metab = [_.id for _ in model.metabolites if _.compartment == compartment[1]]

        for meta in list(metabolites.keys()):
            # get metabolite database annotations
            pos_ids = metab_info.substance_table[metab_info.substance_table['name']==meta][['db_type','db_id']]

            # check namespace availability 
            if not any(namespace in _ for _ in list(pos_ids['db_type'])):
                return None
            # check metabolite availability in model
            id = pos_ids[pos_ids['db_type'].str.contains(namespace)]['db_id']
            for i in id:
                match namespace:
                    # BiGG ID needs the compartment with a '_' as suffix
                    case 'BiGG':
                        if len(metabolites) == 2: # special case proton
                            i_p = i + '_' + compartment[1]
                            i_c = i + '_' + compartment[0]
                            if i_p not in c2_metab or i_c not in c1_metab:
                                return None
                            else:
                                found_ids[i_p] = metabolites['Hydrogen [H(+)] transported']
                                found_ids[i_c] = metabolites['Hydrogen [H(+)]']
                                return found_ids
                        else:
                            i += '_' + compartment[0]
                    # No namespacce given
                    case _:
                        mes = f'Unknown namespace or no namespace given: {namespace}'
                        warnings.warn(mes)
                        return None

                if i not in c1_metab:
                    return None
                else:
                    found_ids[i] = metabolites[meta]
                    break 

        return found_ids

    # retrieve information about dissipation reaction metabolites
    metab_info = Medium('dissipation reaction metabolites')
    metab_info.add_subset('DiReM')
    
    # add dissipations reactions to model
    for name, metabolites in DISSIPATION_RXNS.items():
        if "DISSI_"+name not in model.reactions:
            ids_in_namespace = check_metab_integration(metabolites, model,
                                       metab_info, namespace,compartment)
            if ids_in_namespace:
                rea = cobra.Reaction(name=name, id="DISSI_"+name)
                model.add_reactions([rea])
                rea.add_metabolites(ids_in_namespace)

    return model


def limit_bounds(model: cobra.Model):
    """Limits upper and lower bounds of
    exchange reactions to (0, 0)
    reversible reactions to (-1, 1)
    irreversible reactions to (0, 1)

    excludes dissipation reactions

    Args:
        model (cobra.Model): cobrapy model
    """
    external_comp = cobra.medium.find_external_compartment(model)
    # set fluxes for each reaction within model
    for rea in model.reactions:
        # except dissipation reactions
        if "DISSI_" in rea.id:  
            continue
        # turn off exchange reactions
        elif cobra.medium.is_boundary_type(rea,'exchange',external_comp):
            rea.bounds = (0.0, 0.0)
        # limit reversible reactions to [-1, 1] -> flux 0.1
        elif rea.reversibility: 
            rea.bounds = (-1.0, 1.0)
        # limit irreversible reactions to [0, 1] -> flux 0.1
        else: 
            rea.bounds = (0.0, 1.0)



def find_egcs(model: cobra.Model) -> tuple[dict, dict]:
    """Checks a cobra.Model for 15 Energy Generating Cycles (EGCs) specified in 
    DISSIPATION_RXN by adding the dissipation reactions turning off exchange reactions
    and limit other reactions to 0.1% flux.

    Based on Fritzemaier et al. (2017, https://doi.org/10.1371/journal.pcbi.1005494)    

    Args:
        model (cobra.Model): GEM read in with cobra

    Returns:
        tuple[dict, dict]: (egc_reactions, objective_values)
    """
    # add 15 energy dissipation reactions
    with model as mod_model:  

        # ensure no model modifications
        mod_model = add_DISSIPATIONRXNS(mod_model)

        # set fluxes for each reaction within model
        mod_model = limit_bounds(mod_model)

        # set each dissipation reaction as objective & optimize
        egc_reactions = {}
        obj_vals = {}
        for name in DISSIPATION_RXNS.keys():
            rea_id = "DISSI_"+name
            if rea_id in mod_model.reactions:
                mod_model.objective = "DISSI_"+name
                solution = mod_model.optimize()
                fluxes = solution.fluxes
                objval = solution.objective_value

                if objval > 0.0:  # optimization > 0 --> EGC detected
                    obj_vals[name] = objval
                    egc_reactions[name] = {}

                    for rea, flux in fluxes.items():
                        # cutoff for flux 1.0e-10 (=no growth) 
                        if  (flux > 1.0e-10 or flux < -1.0e-10) and not rea.startswith("DISSI"):
                            egc_reactions[name][rea] = flux

    return (egc_reactions, obj_vals)


#### Identify possible solution
NOTE: based on single changes only -> greedy

In [4]:
def find_egcs_only(model:cobra.Model):

    # add 15 energy dissipation reactions
    with model as mod_model:  

        # ensure no model modifications
        mod_model = add_DISSIPATIONRXNS(mod_model)

        # set fluxes for each reaction within model
        mod_model = limit_bounds(mod_model)

        # set each dissipation reaction as objective & optimize
        egcs = []
        for name in DISSIPATION_RXNS.keys():
            rea_id = "DISSI_"+name
            if rea_id in mod_model.reactions:
                mod_model.objective = "DISSI_"+name
                solution = mod_model.optimize()

                if solution.objective_value > MIN_GROWTH_THRESHOLD:  # optimization > 0 --> EGC detected
                    egcs.append(name)

        return egcs


def egcs_removed(model: cobra.Model, starting_egcs: dict):

    current_egcs = find_egcs_only(model)

    removed = [_ for _ in starting_egcs.keys() if _ not in current_egcs]
    added = [_ for _ in current_egcs if _ not in starting_egcs.keys()]

    if len(added) > 0:
        
        return []
    else: 
        return removed
    

def check_egc_growth(reac: cobra.Reaction, model: cobra.Model, 
                     bounds:tuple ,
                     starting_egcs:dict, threshold:float=MIN_GROWTH_THRESHOLD):

    # set new reaction bounds
    reac.bounds = bounds
    # check egcs removal and growth
    egc_test = egcs_removed(model, starting_egcs)
    if len(egc_test) > 0 and model.optimize().objective_value > threshold:
        return egc_test
    else:
        None


def test_modifications(reaction: cobra.Reaction, model: cobra.Model, 
                       present_egc: dict, threshold:float=MIN_GROWTH_THRESHOLD) -> dict:
    """Tries four cases for a Reaction:
    1. if reaction is not reversible -> make reaction reversible (MR)
    2. limit backward reaction (RB)
    3. limit forward reaction (RF)
    4. "delete" reaction by setting fluxes to 0 (RM)
    -> for each case the EGCs which are present in the model are checked if they are removed
    -> if EGCs are removed we check if the model still grows on optimal medium
    => When both limitations are True reaction is saved to corresponding dictionary

    Args:
        reaction (cobra.Reaction): Reaction from a cobra.Model
        model (cobra.Model): The corresponding GEM loaded with cobrapy
        present_egc (dict): Dictionary of present EGCs {"egc": {}} -> EGCs are keys
        results (dict): "Empty" dictionary of dictionary formated of results {"egc": {"MR":[], "RB":[], "RF":[], "RM":[]}}

    Returns:
        dict: {"egc": {"MR":[potential_solutions],
                       "RB":[potential_solutions],
                       "RF":[potential_solutions],
                       "RM":[potential_solutions]}}
    """
    results = {
            "ATP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "CTP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "GTP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "UTP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "ITP": {"MR": [], "RB": [], "RF": [], "RM": []},
            "NADH": {"MR": [], "RB": [], "RF": [], "RM": []},
            "NADPH": {"MR": [], "RB": [], "RF": [], "RM": []},
            "FADH2": {"MR": [], "RB": [], "RF": [], "RM": []},
            "FMNH2": {"MR": [], "RB": [], "RF": [], "RM": []},
            "Q8H2": {"MR": [], "RB": [], "RF": [], "RM": []},
            "MQL8": {"MR": [], "RB": [], "RF": [], "RM": []},
            "DMMQL8": {"MR": [], "RB": [], "RF": [], "RM": []},
            "ACCOA": {"MR": [], "RB": [], "RF": [], "RM": []},
            "GLU": {"MR": [], "RB": [], "RF": [], "RM": []},
            "PROTON": {"MR": [], "RB": [], "RF": [], "RM": []}
            }
    
    # skip exchange reactions
    if reaction.boundary:
        return None
    
    # test modifications of the reaction
    with model as m:
        original_bounds = reaction.bounds
        # case 1: make reaction reversible
        if not reaction.reversibility:  # skip if reaction is already reversible
            res = check_egc_growth(reaction,m,
                                   (-1000.0,1000.0), # irreversible -> reversible
                                   present_egc,threshold)  
            if res:
                for egc in res:
                    results[egc]["MR"].append(reaction.id)  # MR = make reversible

        # case 2: limit backward reaction
        if original_bounds != (0.0,1000.0):
            res = check_egc_growth(reaction,m,
                                   (0.0,1000.0), # Remove Backward (RB) reaction
                                   present_egc,threshold)  
            if res:
                for egc in res:
                    results[egc]["RB"].append(reaction.id)  

        # case 3: limit forward reaction
        if original_bounds != (-1000.0, 0):
            res = check_egc_growth(reaction,m,
                                   (-1000.0, 0), # Remove Forward (RF) reaction
                                   present_egc,threshold)  
            if res:
                for egc in res:
                    results[egc]["RF"].append(reaction.id)    

        # case 4: "delete" reaction
        if original_bounds != (0.0, 0.0):
            res = check_egc_growth(reaction,m,
                                   (0.0, 0.0),  # Remove (RM) reaction
                                   present_egc,threshold)  
            if res:
                for egc in res:
                    results[egc]["RM"].append(reaction.id)   

    return results


# @TEST good default for limit and chunksize
# params not optimal and runtime options need rechecking
def find_mods_resolve_egcs_greedy(model: cobra.Model, present_egcs: dict,
                                  threshold:float=MIN_GROWTH_THRESHOLD,
                                  limit:int=8, chunksize:int=12) -> dict:
    """Find the (single) modifications to reactions in a cobra.Model and returns these in a dictionary.
    Splits the modification check in multiple processes.
    

    Args:
        model (cobra.Model): input GEM

    Returns:
        dict: Dictionary of potential modifications to resolve EGCs
              {"egc": {"MR":[potential_solutions],
                       "RB":[potential_solutions],
                       "RF":[potential_solutions],
                       "RM":[potential_solutions]}}
    """
    
    output_list = []

    print("_____________________________________")
    print(f"Try to resolve the following EGCs: {[egc for egc in present_egcs]} \nThis might take a while...")

    # try to find BOF
    pos_bofs = test_biomass_presence(model)
    if pos_bofs:
        bof = pos_bofs[0]
    else:
        mes = 'No growth or biomass objectuve function in model. Cannot solve EGCs.'
        raise KeyError(mes)
    
    with model as m:
        set_bounds_to_default(m)
        m.objective = bof

        # might limit processes with Pool(process=limit) -> otherwise it consumes all cores
        # limit to half of machine cores -> at least for me no speed increment with more cores
        
        with Pool(processes=limit) as pool:
                    
            # partial -> creates function with fixed variables to call with each iteration
            # needed since pool.imap cannot do that
            part_test_mods = partial(test_modifications,
                                    model=m,
                                    present_egc=present_egcs,
                                    threshold=threshold)
            # increment in chunksize will reduce computation time -> but progressbar update also...
            for res in list(tqdm(pool.imap_unordered(func=part_test_mods, iterable=m.reactions, 
                                                     chunksize=chunksize),
                            total=len(m.reactions),
                            desc="Resolve EGCs")):
                if res:
                    output_list.append(res)
        
        
    
    # merge output_list to the final results
    results = {}
    for output_dict in output_list:
        for egc, modifications in output_dict.items():
            for mod, reactions in modifications.items():
                if not egc in results.keys():
                    results[egc] = {}
                if mod in results[egc].keys():
                    results[egc][mod] = list(set(results[egc][mod] + reactions))
                else:
                    results[egc][mod] = list(set(reactions))
                

    return results


#### find A solution for the results of `find_mods_resolve_egcs_greedy`

this algorithm is greedy, it does not return the optimal solution

In [5]:
def find_solution_greedy(results):
    
    solved_egcs = set()
    score = 0
    reacs_for_solu = {}

    # make table and drop EGCs that are not in the model
    solution_table = pd.DataFrame(results)[list(egc_reactions.keys())]
    for col in solution_table.columns:
        solution_table[col] = solution_table[col].apply(lambda x: ', '.join(map(str,x)))
    # find not solvable (with the current code) cyles 
    not_solvable = []
    for col in egc_reactions.keys():
        if all(len(_) == 0 for _ in solution_table[col]):
            not_solvable.append(col)
    # report the problems for manual curation
    if len(not_solvable) > 0:
        print(f'The following EGCs cannot be fixed with the current code: {not_solvable}')
        solution_table.drop(not_solvable, axis=1, inplace=True)
    # fix the EGCs that can be fixed with single changes
    solution_table = solution_table.T
    for egc in solution_table.index.to_list():
        # check if sgc has already been taken care of 
        if egc in solved_egcs:
            continue
        
        # best solution for this cycle
        for mode in ECG_SCORING_MATRIX.keys():
            if solution_table.loc[egc, mode]:
                reac = solution_table.loc[egc,mode].split(', ')[0]
                newly_solved = [_ for _ in solution_table.index[solution_table[mode].str.contains(reac)].to_list() if _ not in solved_egcs]
                if len(newly_solved) > 0:
                    solved_egcs = set([*solved_egcs,*newly_solved])
                    score += ECG_SCORING_MATRIX[mode] 
                    reacs_for_solu[reac] = mode

    return (reacs_for_solu, score)


#### Apply solution to model 

In [6]:
def apply_modifications(model: cobra.Model, solution: dict):
    """Apply the modifications to reactions in solution to the model.

    4 modifications are possible:
    "RM" -> removes the reaction
    "RB" -> removes the backwards reaction
    "RF" -> removes the forward reaction
    "MR" -> makes reaction reversible

    Args:
        model (cobra.Model): Input model
        solution (dict): Best solution from calculation in `py:func:find_solution_greedy`.
    """
    for reac, mode in solution.items():
            reaction = model.reactions.get_by_id(reac)
            if type(reaction) is cobra.Reaction:
                match mode:  
                    case "RM":
                        reaction.delete()
                    case "RB":
                        reaction.bounds = (0.0, 1000.0)
                    case "RF":
                        model = reaction.bounds = (1000.0, 0.0)
                    case "MR":
                        model = reaction.bounds = (1000.0, 1000.0)
                    case _:
                        mes = f"{mode} is no viable modification... Something went wrong."
                        warnings.warn(mes)



In [ ]:
def solve_egcs(model:cobra.Model):
    
    egc_reactions, obj_vals = find_egcs(model)
    results = find_mods_resolve_egcs_greedy(model, present_egcs=egc_reactions)
    solution,score = find_solution_greedy(results)
    apply_modifications(model,solution)
    still_egc, vals = find_egcs(model)

    return {'solution':solution, 'score':score, 'remaining egcs':still_egc}

#### Testing

Problem:
currently runtime varies greatly - maybe something to do the the Pool and not reloading the kernel
--> restaring the kernel DEFINITLY improves runtime by a lot

In [1]:
from refinegems.utility.io import load_model
from refinegems.curation.egcs import GreedyEGCSolver

test_egc_path = '/Users/brune/Downloads/iJN746.xml'
print('-')
test_egc = load_model(test_egc_path,'cobra')
print('-')
greedy_solver = GreedyEGCSolver()
greedy_results = greedy_solver.solve_egcs(test_egc)

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


-
-
_____________________________________
Try to resolve the following EGCs: ['ATP', 'CTP', 'GTP', 'UTP', 'NADH', 'NADPH', 'FADH2', 'FMNH2', 'Q8H2', 'ACCOA', 'PROTON'] 
This might take a while...


Resolve EGCs: 100%|██████████| 1054/1054 [01:54<00:00,  9.17it/s]


115.82557320594788

In [7]:
greedy_results['remaining egcs']

[]